# Model serving & deployment testing

## Abstract

This notebook test model deployment using a pre-transformed example.

In [1]:
ENDPOINT_ID='5909171311854223360' 
GOOGLE_CLOUD_PROJECT = 'aliz-ml-spec-2022-submission'
GOOGLE_CLOUD_REGION = 'us-central1'

In [2]:
!gsutil cp -r gs://aliz-ml-spec-2022/demo-1/pipeline_root/taxi-vertex-pipelines/53911330556/taxi-vertex-pipelines-20220621095033/Transform_1699445304064999424/transform_graph/transform_fn transform_fn

Copying gs://aliz-ml-spec-2022/demo-1/pipeline_root/taxi-vertex-pipelines/53911330556/taxi-vertex-pipelines-20220621095033/Transform_1699445304064999424/transform_graph/transform_fn/assets/vocab_compute_and_apply_vocabulary_1_vocabulary...
Copying gs://aliz-ml-spec-2022/demo-1/pipeline_root/taxi-vertex-pipelines/53911330556/taxi-vertex-pipelines-20220621095033/Transform_1699445304064999424/transform_graph/transform_fn/assets/vocab_compute_and_apply_vocabulary_2_vocabulary...
\ [2 files][   53.0 B/   53.0 B]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://aliz-ml-spec-2022/demo-1/pipeline_root/taxi-vertex-pipelines/53911330556/taxi-vertex-pipelines-20220621095033/Transform_1699445304064999424/transform_graph/transform_fn/asset

The block below uses the transform graph output by the Transform component to both preprocess our input data and convert it to the format required by Vertex AI Endpoints.

In [2]:
import tensorflow_transform as tft
import tensorflow as tf

def preprocess_fn(instances):
    loaded_transform_model =tft.TFTransformOutput('transform_fn')
    transform_layer = loaded_transform_model.transform_features_layer()
    raw_data_batch = {
    'dropoff_census_tract': tf.constant([ex['dropoff_census_tract'] for ex in instances], dtype=tf.int64),
    'TripStartHour': tf.constant([ex['TripStartHour'] for ex in instances], dtype=tf.int64),
    'TripStartMinute': tf.constant([ex['TripStartMinute'] for ex in instances], dtype=tf.int64),
    'TripStartYear': tf.constant([ex['TripStartYear'] for ex in instances], dtype=tf.int64),
    'TripStartMonth': tf.constant([ex['TripStartMonth'] for ex in instances], dtype=tf.int64),
    'pickup_census_tract': tf.constant([ex['pickup_census_tract'] for ex in instances], dtype=tf.int64),
    'histOneWeek_tripDistance': tf.constant([ex['histOneWeek_tripDistance'] for ex in instances], dtype=tf.float32),
    'histOneWeek_tripDuration': tf.constant([ex['histOneWeek_tripDuration'] for ex in instances], dtype=tf.float32),
    'historical_tripDuration': tf.constant([ex['historical_tripDuration'] for ex in instances], dtype=tf.float32),
    'historical_tripDistance': tf.constant([ex['historical_tripDistance'] for ex in instances], dtype=tf.float32),
    'rawDistance': tf.constant([ex['rawDistance'] for ex in instances], dtype=tf.float32),
    }
    transformed_input = transform_layer(raw_data_batch)
    for key in transformed_input:
        transformed_input[key] = [transformed_input[key][0][0].numpy().item()]
    return transformed_input

The example below is taken from the dataset we have created previously. The expected output is 40.5.

In [3]:
instance = {'dropoff_census_tract': [17031980100], 'histOneWeek_tripDistance': [12.215],
             'TripStartHour': [16], 'TripStartMinute': [15], 'TripStartYear': [2016],
             'histOneWeek_tripDuration': [3000.0],
             'historical_tripDuration': [2954.0487804878048], 'pickup_census_tract': [17031081500],
             'TripStartMonth': [4], 'historical_tripDistance': [12.332439024390244],
             'rawDistance': [0.16400959606286278]}

In [4]:
from google.cloud import aiplatform


# The AI Platform services require regional API endpoints.
client_options = {
    'api_endpoint': GOOGLE_CLOUD_REGION + '-aiplatform.googleapis.com'
    }
# Initialize client that will be used to create and send requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)


instances = [instance]
transformed_instance = preprocess_fn(instances)

endpoint = client.endpoint_path(
    project=GOOGLE_CLOUD_PROJECT,
    location=GOOGLE_CLOUD_REGION,
    endpoint=ENDPOINT_ID,
)
response = client.predict(endpoint=endpoint, instances=[transformed_instance])
print('prediction:', response.predictions[0])

2022-06-21 11:01:59.309099: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.
prediction: [43.3953552]


The prediction of 43.4 we get back is fairly close to the expected 40.5 from the example